# Huffman Coding

- **Created by Andrés Segura Tinoco**  
- **Created on June 20, 2019**

In computer science and information theory, a **Huffman Code** is a particular type of optimal prefix code that is commonly used for lossless data compression. The output from Huffman's algorithm can be viewed as a variable-length code table for encoding a source symbol (such as a character in a file). The algorithm derives this table from the estimated probability or frequency of occurrence (weight) for each possible value of the source symbol. <a href='#link_one'>[1]</a>

In [1]:
# Load Python libraries
import numpy as np
import timeit
import pandas as pd
from collections import Counter

## 1. Huffman Code from Scratch

In [2]:
# Class HuffmanCode from scratch
class HuffmanCode:
    
    # Return a Huffman code for an ensemble with distribution p
    def get_code(self, p_symbols):
        
        # Init validation
        n = len(p_symbols)
        if n == 0:
            return dict()
        elif n == 1:
            return dict(zip(p_symbols.keys(), ['1']))
        
        # Ensure probabilities sum to 1
        self._normalize_weights(p_symbols)
        
        # Returns Huffman code
        return self._get_code(p_symbols);
    
    # (Private) Calculate Huffman code
    def _get_code(self, p):
        
        # Base case of only two symbols, assign 0 or 1 arbitrarily
        if len(p) == 2:
            return dict(zip(p.keys(), ['0', '1']))
        
        # Create a new distribution by merging lowest prob pair
        p_prime = p.copy()
        s1, s2 = self._get_lowest_prob_pair(p)
        p1, p2 = p_prime.pop(s1), p_prime.pop(s2)
        p_prime[s1 + s2] = p1 + p2
        
        # Recurse and construct code on new distribution
        code = self._get_code(p_prime)
        symbol = s1 + s2
        s1s2 = code.pop(symbol)
        code[s1], code[s2] = s1s2 + '0', s1s2 + '1'
        
        return code;
    
    # Return pair of symbols from distribution p with lowest probabilities
    def _get_lowest_prob_pair(self, p):
        
        # Ensure there are at least 2 symbols in the dist.
        if len(p) >= 2:
            sorted_p = sorted(p.items(), key=lambda x: x[1])
            return sorted_p[0][0], sorted_p[1][0];
        
        return (None, None);
    
    # Makes sure all weights add up to 1
    def _normalize_weights(self, p_symbols, t_weight=1.0):
        n = sum(p_symbols.values())
        
        if n != t_weight:
            for s in p_symbols:
                p_symbols[s] = p_symbols[s] / n;

**Input:**
$$ A = \{ a_1, a_2, a_3, ..., a_n \} \tag{1} $$

$$ W = \{ w_1, w_2, w_3, ..., w_n \} \tag{2} $$

$$ n = |A| $$

**Output:**
$$ C(A, W) = \{ c_1, c_2, c_3, ..., c_n \} \tag{3} $$

**Target:**
$$ L(C) = \sum_{i=1}^n{w_i . length(c_i)} \tag{4} $$

$$ L(C) < L(T)\;for\;any\;code\;T(A, W) $$

In [3]:
# Create Huffman Code instance
hc = HuffmanCode()

### Simple Examples

In [4]:
# Alphabet with 1 symbol
sample_1 = { 'a': 1.0 }
hc.get_code(sample_1)

{'a': '1'}

In [5]:
# Alphabet with 3 symbols and total probability less than 1
sample_2 = { 'a': 0.6, 'b': 0.25, 'c': 0.1 }
hc.get_code(sample_2)

{'a': '0', 'c': '10', 'b': '11'}

In [6]:
# Alphabet with 5 symbols and total probability equal than 1.0
sample_3 = { 'a': 0.10, 'b': 0.15, 'c': 0.30, 'd': 0.16, 'e': 0.29 }
hc.get_code(sample_3)

{'e': '10', 'c': '11', 'd': '00', 'a': '010', 'b': '011'}

## 2. Compress Image with Huffman Code

In [7]:
# Read file in low level (Bytes)
def get_file_bytes(file_path):
    with open(file_path, 'rb') as f:
        return bytearray(f.read());
    return None;

In [8]:
# Loading target image
file_path = "../data/img/example-3.png"
image_byte_list = get_file_bytes(file_path)

In [9]:
# Calculate code frequency
def get_term_freq(term_list):
    term_freq = {}
    terms_count = dict(Counter(term_list))
    
    for key, value in terms_count.items():
        if isinstance(key, int):
            key = chr(key)
        term_freq[key] = value
    
    return term_freq;

In [10]:
# Alphabet with 256 symbols
term_freq = get_term_freq(image_byte_list)
len(term_freq)

256

In [11]:
# Normalize term frequency
n = sum(term_freq.values())
for term in term_freq:
    term_freq[term] = term_freq[term] / n;
sum(term_freq.values())

0.9999999999999994

In [12]:
# Get Huffman coding
hf_code = hc.get_code(term_freq)

In [13]:
# Showing data
codes = pd.DataFrame([term_freq, hf_code]).T
codes.reset_index(level=0, inplace=True)
codes.columns = ["code", "frequency", "binary"]
codes.head(20)

,code,frequency,binary
0, ,0.0284127,01110
1,,0.0197444,110011
2,,0.0157867,100100
3,,0.0114189,000100
4,,0.0123,001110
5,,0.00859678,1010010
6,,0.00885964,1010111
7,,0.00677985,0101111
8,,0.010216,1110001
9,\t,0.00663264,0101011


In [14]:
# Calculate message average size
msg_size_current = 8
msg_size_weighted = 0

for key, value in hf_code.items():
    msg_size_weighted += len(value) * term_freq[key]

In [15]:
# Current message average size (bits per symbol)
msg_size_current

8

In [16]:
# Weighted message average size (bits per symbol)
msg_size_weighted

7.783808280076634

### Real compression percentage

In [17]:
# Calculating compression ratio (%)
compress_rate = (msg_size_current - msg_size_weighted) / msg_size_current
print(round(compress_rate * 100, 2), '%')

2.7 %


#### Main function: compress a binary file using a huffman code

In [18]:
# Compress a binary file using a huffman code
def compress_bin_file(byte_list, code_list):
    compress_list = []
    
    for symbol in byte_list:
        key = chr(symbol)
        new_symbol = code_list[key]
        compress_list.append(new_symbol)
    
    # Return compress file
    return "".join(compress_list)

In [19]:
# Compressing PNG image with Huffman code
compress_file = compress_bin_file(image_byte_list, hf_code)
print(compress_file[:508])

1110101111010001000110101000010111110010001101011000101001101010111001110011101110010000000100100111110111111111111101101110011100110101011100111001110101011100011111100010001000111001110011101111010011000000011010100111001000001110011100111000111000011000101111101111000011011111001110011101010011010100000011000010110110100101010010100100111001110011101100111111101011111110110000101110100000011101110100101111000001011010110110000001110011100001000111010100010011110101111100010010101111110011001100111010


In [20]:
# Weight of the compressed PNG image (KB)
print(round(len(compress_file) / 8 / 1024, 2), 'KB')

451.83 KB


In [21]:
# Weight of the original PNG image (KB)
print(round(len(image_byte_list) / 1024, 2), 'KB')

464.38 KB


## 3. Compress Text file with Huffman Code

In [22]:
# Loading target image
file_path = "../data/text/book-1.txt"
text_byte_list = get_file_bytes(file_path)

In [23]:
# Alphabet with 256 symbols
term_freq = get_term_freq(text_byte_list)
len(term_freq)

110

In [24]:
# Normalize term frequency
n = sum(term_freq.values())
for term in term_freq:
    term_freq[term] = term_freq[term] / n;
sum(term_freq.values())

0.9999999999999998

In [25]:
# Get Huffman coding
hf_code = hc.get_code(term_freq)

In [26]:
# Showing data
codes = pd.DataFrame([term_freq, hf_code]).T
codes.reset_index(level=0, inplace=True)
codes.columns = ["code", "frequency", "binary"]
codes.head(20)

,code,frequency,binary
0,\n,0.0174728,101111
1,\r,0.0174728,101110
2,,0.155075,110
3,!,0.00140562,000010010
4,#,7.95482e-07,11100010100100000110
5,$,1.59096e-06,1110001010010000000
6,&,1.59096e-06,1110001010010000001
7,(,0.000171824,000010011010
8,),0.000171824,000010011011
9,*,6.44341e-05,11100000100111


In [27]:
# Calculate weighted message size average
msg_size_current = 8
msg_size_weighted = 0

for key, value in hf_code.items():
    msg_size_weighted += len(value) * term_freq[key]

In [28]:
# Current message size average (bits per symbol)
msg_size_current

8

In [29]:
# Weighted message size average (bits per symbol)
msg_size_weighted

4.624532355844688

### Real compression percentage

In [30]:
# Calculating compression ratio (%)
compress_rate = (msg_size_current - msg_size_weighted) / msg_size_current
print(round(compress_rate * 100, 2), '%')

42.19 %


In [31]:
# Compressing text file with Huffman code
compress_file = compress_bin_file(text_byte_list, hf_code)
print(compress_file[:508])

1110001010010000010011100010100100000101111000101001001011111011101011110111001010001111111001110010011110101100000100001111110110110101100111001000000000101011110101000011111111101100101110111000110001110000111011001100100001110011010010011011100001100011000001101001111001110011110001010110101000010100011111001101110111001110010100011111110111000000000011000100111111011101110000011010011110111000001111111110111100110000101101110101111111000011001111100110111000001010011100111111101110101111101110101111


In [32]:
# Weight of the compressed text file (KB)
print(round(len(compress_file) / 8 / 1024, 2), 'KB')

709.66 KB


In [33]:
# Weight of the original text file (KB)
print(round(len(text_byte_list) / 1024, 2), 'KB')

1227.64 KB


## 4. Decompress file with Huffman Code

In [34]:
# Compare if two files are equals for equality and element-wise
def compare_files(file_a, file_b):
    return np.array_equiv(file_a, file_b)

#### Main function: decompress a binary file using a huffman code

In [35]:
# Decompress a binary file using a huffman code
def decompress_bin_file(byte_string, code_list):
    start_time = timeit.default_timer()
    byte_list = []
    code_size = []
    inv_codes = {v: k for k, v in hf_code.items()}
    
    for code in inv_codes.keys():
        n = len(code)
        if not n in code_size:
            code_size.append(n)
    code_size.sort()
    
    max_step = max(code_size)
    ix = 0
    n_size = len(byte_string)
    while ix < n_size:
        byte = -1
        
        for size in code_size:
            possible_code = byte_string[ix:ix + size]
            
            if possible_code in inv_codes.keys():
                byte = ord(inv_codes[possible_code])
                ix = ix + size
                break
        
        if byte != -1:
            byte_list.append(byte)
        else:
            print('no exists', ix)
            break
    
    # Elapsed time
    elapsed = timeit.default_timer() - start_time
    print('elapsed time', elapsed, 's')
    
    return byte_list;

In [36]:
# Decode/Decompress file using the Huffman code
decompress_file = decompress_bin_file(compress_file, hf_code)

elapsed time 1.3542215 s


In [37]:
# Weight of the original text file (KB)
print(round(len(decompress_file) / 1024, 2), 'KB')

1227.64 KB


#### Comparing if the original file and the decompressed file are the same (equals)

In [38]:
# Comparing files
compare_files(text_byte_list, decompress_file)

True

## References

<a name='link_one' href='https://en.wikipedia.org/wiki/Huffman_coding' target='_blank' >[1]</a> Wikipedia - Huffman coding.  

<hr>
<p><a href="https://ansegura7.github.io/DataCompression/">« Home</a></p>